# Training neural networks can be very confusing!

What’s a good learning rate? How many hidden layers should your network have? Is dropout actually useful? Why are your gradients vanishing?

In this post we'll peel the curtain behind some of the more confusing aspects of neural nets, and help you make smart decisions about your neural network architecture.

**I highly recommend forking this kernel and playing with the different building blocks to hone your intuition.** I made a quick **demo** to walk you through this kernel: https://www.loom.com/share/fb64035e4576467489cf0f2ad9cff92a. If you have any more questions or feedback, please don't hesitate to [message me](https://twitter.com/lavanyaai)!

## If you like this kernel, please give it an upvote. Thank you! :)

NOTE: Original file was downloaded from here: https://www.kaggle.com/code/lavanyashukla01/training-a-neural-network-start-here under Apache 2.0

Please turn the **'Internet' toggle On** in the Settings panel to your left, in order to make changes to this kernel. You'll be prompted for your Weights and Biases API key, which you can get for free @ [wandb.com](https://www.wandb.com/).

# The Goal

In [1]:
# Please turn the 'Internet' toggle On in the Settings panel to your left, in order to make changes to this kernel.
!pip install wandb -q

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# Essentials
import numpy as np
import pandas as pd

# Models
import tensorflow as tf

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")

# Set random state for numpy
np.random.seed(42)

# WandB
import wandb
from wandb.keras import WandbCallback
# You can change your project name here. For more config options, see https://docs.wandb.com/docs/init.html
wandb.init(anonymous='allow', project="neural-nets-kaggle", name="basic_neural_network")

# Go to https://app.wandb.ai/authorize to get your WandB key

Using TensorFlow backend.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


W&B Run: https://app.wandb.ai/anony-mouse-22574/neural-nets-kaggle/runs/nrrrm339?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [3]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
wandb.init(anonymous='allow', project="building-neural-nets", name="cifar-10")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/wolcx7v6?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [5]:
%%wandb
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, clipnorm=1.0),
              metrics=['accuracy'])

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type="image", labels=labels, validation_data=(x_test, y_test)), keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 20s 407us/step - loss: 1.8189 - accuracy: 0.3325 - val_loss: 1.5400 - val_accuracy: 0.4431
Epoch 2/100
50000/50000 [==============================] - 15s 310us/step - loss: 1.5163 - accuracy: 0.4483 - val_loss: 1.3515 - val_accuracy: 0.5161
Epoch 3/100
50000/50000 [==============================] - 15s 299us/step - loss: 1.3765 - accuracy: 0.5048 - val_loss: 1.2370 - val_accuracy: 0.5577
Epoch 4/100
50000/50000 [==============================] - 15s 300us/step - loss: 1.2796 - accuracy: 0.5435 - val_loss: 1.1682 - val_accuracy: 0.5816
Epoch 5/100
50000/50000 [==============================] - 16s 320us/step - loss: 1.1941 - accuracy: 0.5751 - val_loss: 1.0992 - val_accuracy: 0.6089
Epoch 6/100
50000/50000 [==============================] - 16s 318us/step - loss: 1.1165 - accuracy: 0.6057 - val_loss: 1.0065 - val_accuracy: 0.6461
Epoch 7/100
50000/50000 [=========================

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Training-neural-networks-can-be-very-confusing!" data-toc-modified-id="Training-neural-networks-can-be-very-confusing!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Training neural networks can be very confusing!</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Initialize-Hyperparameters" data-toc-modified-id="Initialize-Hyperparameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialize Hyperparameters</a></span></li><li><span><a href="#Load-dataset" data-toc-modified-id="Load-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load dataset</a></span></li><li><span><a href="#Explore-Dataset" data-toc-modified-id="Explore-Dataset-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Explore Dataset</a></span></li></ul></li><li><span><a href="#Model-Training" data-toc-modified-id="Model-Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model Training</a></span></li><li><span><a href="#Basic-Neural-Network" data-toc-modified-id="Basic-Neural-Network-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Basic Neural Network</a></span></li><li><span><a href="#Learning-Rate" data-toc-modified-id="Learning-Rate-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Learning Rate</a></span></li><li><span><a href="#Momentum" data-toc-modified-id="Momentum-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Momentum</a></span></li><li><span><a href="#Vanishing-+-Exploding-Gradients" data-toc-modified-id="Vanishing-+-Exploding-Gradients-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Vanishing + Exploding Gradients</a></span></li><li><span><a href="#Dropout" data-toc-modified-id="Dropout-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Dropout</a></span></li><li><span><a href="#Optimizers" data-toc-modified-id="Optimizers-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Optimizers</a></span></li><li><span><a href="#Learning-Rate-Scheduling" data-toc-modified-id="Learning-Rate-Scheduling-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Learning Rate Scheduling</a></span></li><li><span><a href="#Make-Predictions" data-toc-modified-id="Make-Predictions-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Make Predictions</a></span></li><li><span><a href="#A-Few-More-Things" data-toc-modified-id="A-Few-More-Things-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>A Few More Things</a></span></li><li><span><a href="#That-was-fun,-yeah?" data-toc-modified-id="That-was-fun,-yeah?-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>That was fun, yeah?</a></span><ul class="toc-item"><li><span><a href="#Weights-&amp;-Biases" data-toc-modified-id="Weights-&amp;-Biases-13.1"><span class="toc-item-num">13.1&nbsp;&nbsp;</span>Weights &amp; Biases</a></span></li></ul></li></ul></div>

## Load dataset

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

# Normalize pixel values
X_train_full, X_test = X_train_full / 255.0, X_test / 255.0

#reshape input data
X_train_full = X_train_full.reshape(X_train_full.shape[0], config.img_width, config.img_height, 1)
X_test = X_test.reshape(X_test.shape[0], config.img_width, config.img_height, 1)

# one hot encode outputs
y_train_full = tf.keras.utils.to_categorical(y_train_full)
y_test = tf.keras.utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# Split into validation, and training sets
X_valid, X_train = X_train_full[:config.validation_size], X_train_full[config.validation_size:]
y_valid, y_train = y_train_full[:config.validation_size], y_train_full[config.validation_size:]
X_train_full.shape, X_train.shape, X_valid.shape, X_test.shape

4423680/4422102 [==============================] - 0s 0us/step


NameError: name 'config' is not defined

## Explore Dataset

![](https://paper-attachments.dropbox.com/s_70ED39A7F97AC42BAB6E155E4809EB5A8988617EE839B00C0D8E924932439DDF_1565465694924_embedding.gif)


Fashion-MNIST is a dataset of that shares the same structure as MNIST (60,000 images in training set, 10,000 in test set, with each image being a 28x28 grayscale image, and 10 label classes to predict). Fashion MNIST is slightly more complicated than MNIST and hence serves as a good dataset for benchmarking machine learning algorithms.

A selection of the images in the dataset are shown below.

# Model Training

# Basic Neural Network

**1. Input neurons**
The input vector needs one input neuron per feature. For tabular data, this is the number of relevant features in your dataset. You want to carefully select these features and remove any that may contain patterns that won't generalize beyond the training set (and cause overfitting). For images, this is the dimensions of your image (28*28=784 in case of MNIST).

        
**2. Output neurons**
This is the number of predictions you want to make.
- **Regression:** For regression tasks, this can be one value (e.g. housing price). For multi-variate regression, it is one neuron per predicted value (e.g. for bounding boxes it can be 4 neurons – one each for bounding box height, width, x-coordinate, y-coordinate).
- **Classification:** For binary classification (spam-not spam), we use one output neuron per positive class, wherein the output represents the probability of the positive class. For multi-class classification (e.g. in object detection where an instance can be classified as a car, a dog, a house etc.), we have one output neuron per class, and use the softmax activation function on the output layer to ensure the final probabilities sum to 1.
        
        
**3. Hidden Layers and Neurons per Hidden Layers**
    The number of hidden layers is highly dependent on the problem and the architecture of your neural network. You're essentially trying to Goldilocks your way into the perfect neural network architecture - not too big, not too small, just right.
        
Generally, 1-5 hidden layers will serve you well for most problems. When working with image or speech data, you'd want your network to have dozens-hundreds of layers, not all of which might be fully connected. For these use cases, there are pre-trained models (YOLO, ResNet, VGG) that allow you to use large parts of their networks, and train your model on top of these networks to learn only the higher order features. In this case, your model will still have only a few layers to train.

In general using the same number of neurons for all hidden layers will suffice. For some datasets, having a large first layer and following it up with smaller layers will lead to better performance as the first layer can learn a lot of lower-level features that can feed into a few higher order features in the subsequent layers.

Usually you will get more of a performance boost from adding more layers than adding more neurons in each layer.

I'd recommend starting with 1-5 layers and 1-100 neurons and slowly adding more layers and neurons until you start overfitting. You can track your loss and accuracy within your Weights and Biases dashboard to see which hidden layers + hidden neurons combo leads to the best loss.

Something to keep in mind with choosing a smaller number of layers/neurons is that if the this number is too small, your network will not be able to learn the underlying patterns in your data and thus be useless. An approach to counteract this is to start with a huge number of hidden layers + hidden neurons and then use dropout and early stopping to let the neural network size itself down for you. Again, I'd recommend trying a few combinations and track the performance in your Weights and Biases dashboard to determine the perfect network size for your problem.

Andrej Karpathy also recommends the [overfit then regularize](http://karpathy.github.io/2019/04/25/recipe/) approach – "first get a model large enough that it can overfit (i.e. focus on training loss) and then regularize it appropriately (give up some training loss to improve the validation loss)."

        
**4. Loss function**
![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565307636338_image.png)
   - **Regression:** Mean squared error is the most common loss function to optimize for, unless there are a significant number of outliers. In this case, use mean absolute error or Huber loss.     
   - **Classification:** Cross-entropy will serve you well in most cases.


**5. Batch Size**
Large batch sizes can be great because they can harness the power of GPUs to process more training instances per time. [OpenAI has found](https://openai.com/blog/science-of-ai/) larger batch size (of tens of thousands for image-classification  and  language modeling, and of millions in the case of RL agents) serve well for scaling and parallelizability.

There's a case to be made for smaller batch sizes too, however. According to [this paper](https://arxiv.org/abs/1804.07612) by Masters and Luschi, the advantage gained from increased parallelism from running large batches is offset by the increased performance generalization and smaller memory footprint achieved by smaller batches. They show that increased batch sizes reduce the acceptable range of learning rates that provide stable convergence. Their takeaway is that smaller is, in-fact, better; and that the best performance is obtained by mini-batch sizes between 2 and 32.

If you're not operating at massive scales, I would recommend starting with lower batch sizes and slowly increasing the size and monitoring performance in your [Weights and Biases](https://www.wandb.com/) dashboard to determine the best fit.
        
**6. Number of epochs**
    I'd recommend starting with a large number of epochs and use early stopping to halt training when performance stops improving.
        
        
**7. Scaling your features**
    A quick note: Make sure all your features have similar scale before using them as inputs to your neural network. This ensures faster convergence. When your features have different scales (e.g. salaries in thousands and years of experience in tens), the cost function will look like the elongated bowl on the left. This means your optimization algorithm will take a long time to traverse the valley compared to using normalized features (on the right).
    ![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565307687940_image.png)
        

In [7]:
config = wandb.config # Config is a variable that holds and saves hyperparameters and inputs
config.dropout = 0.2
config.conv_layer_1_size  = 32
config.conv_layer_2_size = 64
config.conv_layer_3_size = 128
config.hidden_layer_size = 512
config.learn_rate = 0.01
config.learn_rate_low = 0.001
config.kernel_size = 3
config.pool_size = 2
config.decay = 1e-6
config.momentum = 0.9
config.n_epochs = 25

config.img_width=28
config.img_height=28
config.num_classes = 10
config.batch_size = 128
config.validation_size = 5000

In [8]:
%%wandb 
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu', input_shape=(config.img_width, config.img_height, 1)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='relu'))
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=config.learn_rate), metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_valid, y_valid), epochs=config.n_epochs,
        callbacks=[WandbCallback(data_type="image", validation_data=(X_valid, y_valid), labels=labels)])

NameError: name 'X_train' is not defined

# Learning Rate



![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565307718429_image.png)

Picking the learning rate is very important, and you want to make sure you get this right! Ideally you want to re-tweak the learning rate when you tweak the other hyper-parameters of your network.
    
To find the best learning rate, start with a very low values (10^-6) and slowly multiply it by a constant until it reaches a very high value (e.g. 10). Measure your model performance (vs the log of your learning rate) in your Weights and Biases dashboard to determine which rate served you well for your problem. You can then retrain your model using this optimal learning rate.
    
The best learning rate is usually half of the learning rate that causes the model to diverge. Feel free to set different values for learn_rate in the code and seeing how it affects model performance to develop your intuition around learning rates.

I'd also recommend using the [Learning Rate finder](https://arxiv.org/abs/1506.01186) method proposed by Leslie Smith. It an excellent way to find a good learning rate for most gradient optimizers (most variants of SGD) and works with most network architectures.
    
Also, see the section on learning rate scheduling below.

In [9]:
wandb.init(anonymous='allow', project="building-neural-nets", name="lower_learning_rate")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/vodysraz?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [10]:
%%wandb
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu', input_shape=(config.img_width, config.img_height, 1)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='relu'))
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=config.learn_rate_low), metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_valid, y_valid), epochs=config.n_epochs,
        callbacks=[WandbCallback(data_type="image", validation_data=(X_valid, y_valid), labels=labels)])

NameError: name 'X_train' is not defined

# Momentum
![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565307746069_image.png)

Gradient Descent takes tiny, consistent steps towards the local minima and when the gradients are tiny it can take a lot of time to converge. Momentum on the other hand takes into account the previous gradients, and accelerates convergence by pushing over valleys faster and avoiding local minima.
    
In general you want your momentum value to be very close to one. 0.9 is a good place to start for smaller datasets, and you want to move progressively closer to one (0.999) the larger your dataset gets. (Setting nesterov=True lets momentum take into account the gradient of the cost function a few steps ahead of the current point, which makes it slightly more accurate and faster.)

In [11]:
wandb.init(anonymous='allow', project="building-neural-nets", name="momentum")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/9y6q29fk?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [12]:
%%wandb
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu', input_shape=(config.img_width, config.img_height, 1)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='relu'))
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=config.learn_rate_low, decay=config.decay, momentum=config.momentum,
                            nesterov=True), metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_valid, y_valid), epochs=config.n_epochs,
        callbacks=[WandbCallback(data_type="image", validation_data=(X_valid, y_valid), labels=labels)])

NameError: name 'X_train' is not defined

# Vanishing + Exploding Gradients

![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565308012033_vanishing_gradients.gif)

Just like people, not all neural network layers learn at the same speed. So when the backprop algorithm propagates the error gradient from the output layer to the first layers, the gradients get smaller and smaller until they're almost negligible when they reach the first layers. This means the weights of the first layers aren't updated significantly at each step.
    
This is the problem of vanishing gradients. (A similar problem of exploding gradients occurs when the gradients for certain layers get progressively larger, leading to massive weight updates for some layers as opposed to the others.)
    
There are a few ways to counteract vanishing gradients. Let's take a look at them now!
    
**1. Activation functions (non-saturating functions)**
- **Hidden Layer Activation**
    In general, the performance from using different [activation functions](https://isaacchanghau.github.io/post/activation_functions/) improves in this order (from lowest→highest performing): logistic → tanh → ReLU → Leaky ReLU → ELU → SELU. ReLU is the most popular activation function and if you don't want to tweak your activation function, ReLU is a great place to start. But, keep in mind ReLU is becoming increasingly less effective than [ELU](https://arxiv.org/pdf/1511.07289.pdf) or [GELU](https://arxiv.org/pdf/1606.08415.pdf). If you’re feeling more adventurous, you can try the following:
    - to combat neural network overfitting: RReLU
    - reduce latency at runtime: leaky ReLU
    - for massive training sets: PReLU
    - for fast inference times: leaky ReLU
    - if your network doesn't self-normalize: ELU
    - for an overall robust activation function: SELU
    
    [This](https://arxiv.org/pdf/1811.03378.pdf) is an excellent paper that dives deeper into the comparison of various activation functions for neural networks.

    As always, don't be afraid to experiment with a few different activation functions, and turn to your Weights and Biases dashboard to help you pick the one that works best for you!    

- **Output Layer Activation**
    - **Regression:** Regression problems don't require activation functions for their output neurons because we want the output to take on any value. In cases where we want out values to be bounded into a certain range, we can use tanh for -1→1 values and logistic function for 0→1 values. In cases where we're only looking for positive output, we can use softplus activation.

    - **Classification:** Use the sigmoid activation function for binary classification to ensure the output is between 0 and 1. Use softmax for multi-class classification to ensure the output probabilities add up to 1.
            
            
**2. Weight initialization method**
    The right weight initialization method can speed up time-to-convergence considerably. The choice of your initialization method depends on your activation function. Some things to try:
- When using ReLU or leaky RELU, use [He initialization](https://arxiv.org/pdf/1502.01852.pdf)
- When using SELU or ELU, use [LeCun initialization](http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf)
- When using softmax, logistic, or tanh, use [Glorot initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Most initialization methods come in uniform and normal distribution flavors.
        
        
**3. BatchNorm**
BatchNorm simply learns the optimal means and scales of each layer's inputs. It does so by zero-centering and normalizing its input vectors, then scaling and shifting them. It also acts like a regularizer which means we don't need dropout or L2 reg.

Using BatchNorm lets us use larger learning rates (which result in faster convergence) and lead to huge improvements in most neural networks by reducing the vanishing gradients problem. The only downside is that it slightly increases training times because of the extra computations required at each layer.
        
        
**4. Gradient Clipping**
A great way to reduce gradients from exploding, specially when training RNNs, is to simply clip them when they exceed a certain value. I'd recommend trying clipnorm instead of clipvalue, which allows you to keep the direction of your gradient vector consistent. Clipnorm contains any gradients who's l2 norm is greater than a certain threshold.

Try a few different threshold values to find one that works best for you.
        
        
**5. Early Stopping**
Early Stopping lets you live it up by training a model with more hidden layers, hidden neurons and for more epochs than you need, and just stopping training when performance stops improving consecutively for n epochs. It also saves the best performing model for you. You can enable Early Stopping by setting up a callback when you fit your model and setting `save_best_only=True`.

In [13]:
wandb.init(anonymous='allow', project="building-neural-nets", name="vanishing_gradients")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/akfwszdb?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [14]:
%%wandb
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal',
                 input_shape=(config.img_width, config.img_height, 1)))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=config.learn_rate, clipnorm=1.0),
              metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_test, y_test), epochs=config.n_epochs,
          callbacks=[WandbCallback(data_type="image", validation_data=(X_valid, y_valid), labels=labels), tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

NameError: name 'X_train' is not defined

# Dropout

Dropout is a fantastic regularization technique that gives you a massive performance boost (~2% for state-of-the-art models) for how simple the technique actually is. All dropout does is randomly turn off a percentage of neurons at each layer, at each training step. This makes the network more robust because it can't rely on any particular set of input neurons for making predictions. The knowledge is distributed amongst the whole network. Around 2^n (where n is the number of neurons in the architecture) slightly-unique neural networks are generated during the training process, and ensembled together to make predictions.

A good dropout rate is between 0.1 to 0.5; 0.3 for RNNs, and 0.5 for CNNs. Use larger rates for bigger layers. Increasing the dropout rate decreases overfitting, and decreasing the rate is helpful to combat under-fitting.

You want to experiment with different rates of dropout values, in earlier layers of your network, and check your Weights and Biases dashboard to pick the best performing one. You definitely don’t want to use dropout in the output layers.

Read [this paper](https://arxiv.org/abs/1801.05134) before using Dropout in conjunction with BatchNorm.

In this kernel I used AlphaDropout, a flavor of the vanilla dropout that works well with SELU activation functions by preserving the input's mean and standard deviations.

In [15]:
wandb.init(anonymous='allow', project="building-neural-nets", name="dropout")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/javxlfeh?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [16]:
%%wandb
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal',
                 input_shape=(config.img_width, config.img_height, 1)))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=config.learn_rate, clipnorm=1.0),
              metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_test, y_test), epochs=config.n_epochs,
          callbacks=[WandbCallback(data_type="image", validation_data=(X_valid, y_valid), labels=labels), tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

NameError: name 'X_train' is not defined

# Optimizers


Gradient Descent isn't the only optimizer game in town! There's a few different ones to choose from. This post does a good job of describing some of the optimizers you can choose from.
![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565307886324_optimization_algorithms.gif)

My general advice is to use Stochastic Gradient Descent if you care deeply about quality of convergence and if time is not of the essence.

If you care about time-to-convergence and a point close to optimal convergence will suffice, experiment with Adam, Nadam, RMSProp, and Adamax optimizers. You Weights and Biases dashboard will guide you to the optimizer that works best for you!

Adam/Nadam are usually good starting points, and tend to be quite forgiving to a bad learning late and other non-optimal hyperparameters.

[According to Andrej Karpathy](http://karpathy.github.io/2019/04/25/recipe/), "a well-tuned SGD will almost always slightly outperform Adam" in the case of ConvNets.

In this kernel, I got the best performance from Nadam, which is just your regular Adam optimizer with the Nesterov trick, and thus converges faster than Adam.

In [17]:
wandb.init(anonymous='allow', project="building-neural-nets", name="nadam_optimizer")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/q3u2j9me?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [18]:
%%wandb
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal',
         input_shape=(config.img_width, config.img_height, 1)))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Nadam(lr=config.learn_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0), metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_test, y_test), epochs=config.n_epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels, validation_data=(X_valid, y_valid)), tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

NameError: name 'X_train' is not defined

# Learning Rate Scheduling


![](https://paper-attachments.dropbox.com/s_39292DB9CE2A9400103E176C2ABC438C6A626910E9DBB0D6FBE28EE673C7492C_1565307910210_image.png)

We talked about the importance of a good learning rate already – we don't want it to be too high, lest the cost function dance around the optimum value and diverge. We also don't want it to be too low because that means convergence will take a very long time.

Babysitting the learning rate can be tough because both higher and lower learning rates have their advantages. The great news is that we don't have to commit to one learning rate! With learning rate scheduling we can start with higher rates to move faster through gradient slopes, and slow it down when we reach a gradient valley in the hyper-parameter space which requires taking smaller steps.

There are many ways to schedule learning rates including decreasing the learning rate exponentially, or by using a step function, or tweaking it when the performance starts dropping, or using 1cycle scheduling. In this kernel, I show you how to use the ReduceLROnPlateau callback to reduce the learning rate by a constant factor whenever the performance drops for n epochs.

I would highly recommend also trying out 1cycle scheduling.

Use a constant learning rate until you've trained all other hyper-parameters. And implement learning rate decay scheduling at the end.

As with most things, I'd recommend running a few different experiments with different scheduling strategies and using your Weights and Biases dashboard to pick the one that leads to the best model.

In [19]:
wandb.init(anonymous='allow', project="building-neural-nets", name="learningrate")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/ip3oroi9?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [20]:
%%wandb
# build model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal',
         input_shape=(config.img_width, config.img_height, 1)))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_1_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D((config.pool_size, config.pool_size)))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.Conv2D(config.conv_layer_2_size, kernel_size=(config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
model.add(tf.keras.layers.MaxPooling2D(pool_size=(config.pool_size, config.pool_size)))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Conv2D(config.conv_layer_3_size, (config.kernel_size, config.kernel_size), activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.hidden_layer_size, activation='selu', kernel_initializer='lecun_normal'))
tf.keras.layers.BatchNormalization(),
tf.keras.layers.AlphaDropout(rate=config.dropout),
model.add(tf.keras.layers.Dense(config.num_classes, activation='softmax'))

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Nadam(lr=config.learn_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0), metrics=['accuracy'])
model.fit(X_train, y_train, verbose=0, validation_data=(X_test, y_test), epochs=config.n_epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels, validation_data=(X_valid, y_valid)), tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True), lr_scheduler])

NameError: name 'X_train' is not defined

# Make Predictions

In [21]:
model.evaluate(X_test, y_test)

ValueError: Error when checking input: expected conv2d_30_input to have 4 dimensions, but got array with shape (10000, 28, 28)

In [22]:
model.save("fashion_mnist_model.h5")

# Let's make things more interesting - CIFAR100

In [23]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [24]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [25]:
wandb.init(anonymous='allow', project="building-neural-nets", name="cifar-10")

W&B Run: https://app.wandb.ai/anony-mouse-22574/building-neural-nets/runs/vid02cse?apiKey=d380db39032b322bb131f39a0d30e0f2bfb6f899

In [26]:
%%wandb
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, clipnorm=1.0),
              metrics=['accuracy'])

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type="image", labels=labels, validation_data=(x_test, y_test)), keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 16s 329us/step - loss: 1.8276 - accuracy: 0.3276 - val_loss: 1.5199 - val_accuracy: 0.4521
Epoch 2/100
50000/50000 [==============================] - 16s 315us/step - loss: 1.5054 - accuracy: 0.4533 - val_loss: 1.4504 - val_accuracy: 0.4787
Epoch 3/100
50000/50000 [==============================] - 16s 327us/step - loss: 1.3744 - accuracy: 0.5035 - val_loss: 1.2411 - val_accuracy: 0.5632
Epoch 4/100
50000/50000 [==============================] - 16s 317us/step - loss: 1.2743 - accuracy: 0.5474 - val_loss: 1.1443 - val_accuracy: 0.5981
Epoch 5/100
50000/50000 [==============================] - 15s 308us/step - loss: 1.1918 - accuracy: 0.5773 - val_loss: 1.0703 - val_accuracy: 0.6245
Epoch 6/100
50000/50000 [==============================] - 16s 313us/step - loss: 1.1169 - accuracy: 0.6035 - val_loss: 1.0231 - val_accuracy: 0.6432
Epoch 7/100
50000/50000 [=========================

# A Few More Things
- Try [EfficientNets](https://arxiv.org/pdf/1905.11946.pdf) to scale your network in an optimal way.
- Read [this paper](https://arxiv.org/pdf/1803.09820.pdf) for an overview of some additional learning rate, batch sizes, momentum and weight decay techniques.
- And [this one](https://arxiv.org/abs/1803.05407) on Stochastic Weight Averaging (SWA). It shows that better generalization can be achieved by averaging multiple points along the SGD's trajectory, with a cyclical or constant learning rate.
- Read Andrej Karpathy's [excellent guide](http://karpathy.github.io/2019/04/25/recipe/) on getting the most juice out of your neural networks.

## If you like this kernel, please give it an upvote. Thank you! :)

# That was fun, yeah?

We've explored a lot of different facets of neural networks in this post!

We've looked at how to setup a basic neural network (including choosing the number of hidden layers, hidden neurons, batch sizes etc.)

We've learnt about the role momentum and learning rates play in influencing model performance.

And finally we've explored the problem of vanishing gradients and how to tackle it using non-saturating activation functions, BatchNorm, better weight initialization techniques and early stopping.

You can compare the accuracy and loss performances for the various techniques we tried in one single chart, by visiting your [Weights and Biases](https://app.wandb.ai/home) dashboard.

Neural networks are powerful beasts that give you a lot of levers to tweak to get the best performance for the problems you're trying to solve! The sheer size of customizations that they offer can be overwhelming to even seasoned practitioners. Tools like Weights and Biases are your best friends in navigating the land of the hyper-parameters, trying different experiments and picking the most powerful models.

I hope this guide will serve as a good starting point in your adventures.

If you have any questions or feedback, please don't hesitate to [message me](https://twitter.com/lavanyaai)!

Good luck!


----------
## Weights & Biases

We're building lightweight, flexible experiment tracking tools for deep learning. Add a couple of lines to your python script, and we'll keep track of your hyperparameters and output metrics, making it easy to compare runs and see the whole history of your progress. Think of us like GitHub for deep learning. Find out more at [wandb.com](http://wandb.com).
